In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
object_dict = pickle.load(open("flame_mechanism_20250228_1.pkl",'rb'))
print(object_dict.keys())
flame_temperature_3D_array = object_dict["flame_temperature_3D_array"]
flame_speed_3D_array = object_dict["flame_speed_3D_array"]
Phi_array = object_dict["Phi_array"]
T_array = object_dict["T_array"]
P_array = object_dict["P_array"]
print(flame_temperature_3D_array.shape)
print(flame_speed_3D_array.shape)
print(Phi_array.shape)
print(T_array.shape)
print(P_array.shape)
print(object_dict["metadata"])
print("Phi_array", Phi_array)
print("T_array", T_array)
print("P_array", P_array)


dict_keys(['flame_temperature_3D_array', 'flame_speed_3D_array', 'Phi_array', 'T_array', 'P_array', 'metadata'])
(22, 15, 4)
(22, 15, 4)
(22,)
(15,)
(4,)
Flame temperature and speed values at grid points.         The values are interpolated from flame simulation using `phi`, `T`, and `P`.         The values of `phi`, `T`, `P` grid points are included in the dict.
Phi_array [0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.72 0.74 0.76 0.78 0.8  0.82 0.84
 0.86 0.88 0.9  0.92 0.94 0.96 0.98 1.  ]
T_array [ 6  8 10 12 14 16 18 20 22 24 26 28 30 32 34]
P_array [0.9  0.95 1.   1.05]


In [3]:
from scipy.interpolate import LinearNDInterpolator

# 创建网格点的坐标矩阵
T_grid, P_grid, Phi_grid = np.meshgrid(T_array, P_array, Phi_array, indexing='ij')
points = np.array([T_grid.ravel(), P_grid.ravel(), Phi_grid.ravel()]).T

# 创建插值函数
flame_temperature_interpolator = LinearNDInterpolator(points, flame_temperature_3D_array.ravel()) # 不规则网格线性插值
flame_speed_interpolator = LinearNDInterpolator(points, flame_speed_3D_array.ravel()) # 不规则网格线性插值


In [4]:
file_paths = ["20250118_1.xlsx", "20250118_2.xlsx", "20250118_3.xlsx", "20250118_4.xlsx"]
data_list = []

for i, file_path in enumerate(file_paths):
    df = pd.read_excel(file_path, sheet_name='Sheet1', header=0)
    df=df.drop(columns=['Unnamed: 0'])
    df=df.interpolate()
    
    import cantera as ct
    recon_flame_temperature = []
    recon_flame_speed = []
    for index, row in df.iterrows():
        T = row['Environment Temperature']
        P = row['Inside Pressure']*1000/ct.one_atm
        O2 = row['O2']*0.01
        Phi = (0.21-O2)/(0.21-0.1015355*O2)
        query_points = np.array([[T, P, Phi]])
        recon_flame_temperature.append(flame_temperature_interpolator(query_points).item())
        recon_flame_speed.append(flame_speed_interpolator(query_points).item())
    df['Flame Temperature'] = recon_flame_temperature
    df['Flame Speed'] = recon_flame_speed
    df=df.interpolate()
    data_list.append(df.to_numpy())
    print(f"File {i} information:")
    df.info()


File 0 information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3349 entries, 0 to 3348
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   O2                       3349 non-null   float64
 1   CO2                      3349 non-null   float64
 2   NOx                      3349 non-null   float64
 3   ppm CO                   3349 non-null   int64  
 4   ppm NO                   3349 non-null   int64  
 5   ppm NO2                  3349 non-null   float64
 6   Smoke Temperature        3349 non-null   float64
 7   Environment Temperature  3349 non-null   float64
 8   Pumping Flow             3349 non-null   float64
 9   Inside Temperature       3349 non-null   float64
 10  Inside Humidity          3349 non-null   float64
 11  Inside Pressure          3349 non-null   float64
 12  Outside Temperature      3349 non-null   float64
 13  Outside Humidity         3349 non-null   float64
 14  Outs

In [ ]:
pickle.dump(data_list, open('data1344.pkl', 'wb'))